In [1]:
import torch
import numpy as np
import pandas as pd
import copy
import torch.nn.functional as F
import scipy.sparse as sp
from torch_geometric.nn import GCNConv,GATConv,SAGEConv
from torch_geometric.datasets import Planetoid

In [2]:
# 把标签转换成onehot
def encode_onehot(labels):                                   
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
    return labels_onehot

In [4]:
# 测试onehot函数
labels = ("input","output","entityFunctionalStatus","catalystActivity","regulatedBy")
a = encode_onehot(labels)
print (a)

[[1 0 0 0 0]
 [0 0 0 0 1]
 [0 0 0 1 0]
 [0 1 0 0 0]
 [0 0 1 0 0]]


In [3]:
def normalize(mx):                                          # 归一化
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten() ## 幂
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv) # 对角化
    mx = r_mat_inv.dot(mx) # 乘积
    mx = mx * 10000
    return mx

In [79]:
# 测试normailize 函数
mx1 =  np.arange(0, 30, 2).reshape(3,5).astype(np.float32)
print (mx1)
rowsum = np.array(mx1.sum(1))
print (rowsum)
r_inv = np.power(rowsum, -1).flatten()
print (r_inv)
print (normalize(mx1))
#a = torch.rand(2,3,4)
#print (a)

[[ 0.  2.  4.  6.  8.]
 [10. 12. 14. 16. 18.]
 [20. 22. 24. 26. 28.]]
[ 20.  70. 120.]
[0.05       0.01428571 0.00833333]
[[0.         0.1        0.2        0.3        0.4       ]
 [0.14285715 0.17142858 0.2        0.22857143 0.25714287]
 [0.16666667 0.18333334 0.20000002 0.21666668 0.23333335]]


In [4]:
GTEx_exp = pd.read_csv("/public/home/liujunwu/workdir/scripts/GNN_Reactome/GTEx_exp/GTEx_exp.csv.test",sep=',',header=0)
print (GTEx_exp.iloc[0:3,0:3])

                     Sample          Tissue  DDX11L1
0  GTEX-1117F-0226-SM-5GZZ7  Adipose Tissue      0.0
1  GTEX-1117F-0426-SM-5EGHI          Muscle      0.0
2  GTEX-1117F-0526-SM-5EGHJ    Blood Vessel      0.0


In [5]:
#print (GTEx_exp.iloc[0:3,-2:-1])
# log2表达量归一化
normlizeGTEx = pd.DataFrame(normalize(GTEx_exp.iloc[:,2:GTEx_exp.shape[1]]),columns=GTEx_exp.iloc[:,2:GTEx_exp.shape[1]].columns)
#print (pd.DataFrame(normlizeGTEx).shape)
print (normlizeGTEx.iloc[0:3,[0,1,-4]])
exp_dict=normlizeGTEx.to_dict('dict')

   DDX11L1    WASH7P     MT-TE
0      0.0  0.443207  0.553907
1      0.0  0.443591  1.069985
2      0.0  0.442149  0.668421


In [6]:
print (exp_dict['WASH7P'][1])

0.44359060869091066


In [231]:
print (normlizeGTEx.shape)
del GTEx_exp

(99, 54393)


In [23]:
# 整理蛋白和基因名的对应关系
reaction_protein = pd.read_csv("/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/reaction_related.protein",sep='\t',header=None)
reaction_protein_list = reaction_protein.loc[:,0].tolist()
#reaction_protein.columns = ["Protein"] ## 9233 proteins, 其表达量作为节点reaction的特征
P2Symbol = pd.read_csv("/public/ref/UniProt/HUMAN_9606_id.GeneName",sep='\t',header=None)
P2Symbol.columns = ["Protein","Genecard","GeneSymbol"]
P2Symbol_dict = P2Symbol.set_index(['Protein'])['GeneSymbol'].to_dict()
reaction_gene_list = []
for p in reaction_protein_list:
    if (p in P2Symbol_dict.keys()):
        reaction_gene_list.append(P2Symbol_dict[p])
    else:
        #reaction_gene_list.append(p) # 是否保留找不到基因名的蛋白
        pass
reaction_nodes = pd.read_csv("/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/reactome_reaction.nodes.txt",sep='\t',header=0)
print (reaction_nodes.shape)
reaction_nodes.columns = ["Reaction","Type"]
zero_matrix = pd.DataFrame(np.zeros((reaction_nodes.shape[0],len(reaction_gene_list))),columns = reaction_gene_list)
print (zero_matrix.shape)
print (reaction_nodes.iloc[0:3,0:3])
print (zero_matrix.iloc[0:3,0:3])
nodes_init_matrix = pd.concat([reaction_nodes,zero_matrix],axis=1)
print (nodes_init_matrix.iloc[0:10,0:3])
#reaction_edges = "/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/reactome_reaction.edges.txt"
## 根据表达量生成 样本的nodes_feature 
print (nodes_init_matrix.loc[0,"Reaction"])

(13589, 2)
(13589, 9029)
        Reaction             Type
0  R-HSA-5216072     EwasReaction
1  R-HSA-5226964  ProteinReaction
2  R-HSA-5215980     EwasReaction
   IGKV2-28  IGKV2D-30  TRAV19
0       0.0        0.0     0.0
1       0.0        0.0     0.0
2       0.0        0.0     0.0
        Reaction             Type  IGKV2-28
0  R-HSA-5216072     EwasReaction       0.0
1  R-HSA-5226964  ProteinReaction       0.0
2  R-HSA-5215980     EwasReaction       0.0
3  R-HSA-5339528  ProteinReaction       0.0
4  R-HSA-5339529     EwasReaction       0.0
5  R-HSA-5336453  ProteinReaction       0.0
6  R-HSA-5336426             None       0.0
7  R-HSA-5336454  ProteinReaction       0.0
8  R-HSA-5216137     EwasReaction       0.0
9  R-HSA-8932851  ProteinReaction       0.0
R-HSA-5216072


In [30]:
print (nodes_init_matrix.shape)
common_gene_list = set(reaction_gene_list) & set(GTEx_exp.columns)
GTEx_exp_match = GTEx_exp.loc[:,list(common_gene_list)]
print (GTEx_exp_match.shape)

(13589, 9031)
(99, 8832)


In [44]:
## 获取每个样本的节点特征矩阵
protein_reaction_file= '/public/ref/Msigdb/Reactome/useInfo/Human.ProteinRoleReaction.txt'
prf = pd.read_csv(protein_reaction_file,sep='\t',header=None)
prf.columns = ["Protein",'Realation','Reaction']
prf["Gene"] = prf['Protein'].apply(lambda x : P2Symbol_dict[x] if x in P2Symbol_dict.keys() else x)
reaction_gene_dict = {}
reaction_gene_list = []
print (prf.loc[prf["Reaction"] == 'R-HSA-5226964'])
print (type(prf.loc[2,"Reaction"]))
for i in range(prf.shape[0]):
    reaction_gene_dict.setdefault(prf.loc[i,"Reaction"],[]).append(prf.loc[i,"Gene"]) ## Reaction包含哪些基因,重复的也会append
    #reaction_gene_dict[prf.loc[i,"Reaction"]] = list(set(reaction_gene_dict[prf.loc[i,"Reaction"]]))
print (len(reaction_gene_dict))
#for i, (k, v) in enumerate(reaction_gene_dict.items()):
#    if i in range(0, 3):
#        print(k, v)
print (reaction_gene_dict["R-HSA-5336453"])
print (reaction_gene_dict["R-HSA-5226964"])

       Protein         Realation       Reaction  Gene
194943  Q9HCJ1  catalystActivity  R-HSA-5226964  ANKH
<class 'str'>
13702
['NIPAL3', 'NIPAL2', 'NIPAL4', 'NIPA2', 'NIPA1', 'NIPAL1']
['ANKH']


In [49]:
import datetime
nodes_features = list()
#print (type(GTEx_exp))
print (reaction_gene_dict["R-HSA-5226964"])
print ("ANKH" in normlizeGTEx.columns)
print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))
for index in range(GTEx_exp_match.shape[0]):
    print (index)
    sample_node_feature = copy.deepcopy(nodes_init_matrix)
    for j in range(sample_node_feature.shape[0]):
        if (sample_node_feature.loc[j,"Type"] != "ProteinReaction"):continue
        reaction_name = sample_node_feature.loc[j,"Reaction"]
        #print (j)
        #print (reaction_name)
        if reaction_name in reaction_gene_dict.keys():
            hsa_gene_list = reaction_gene_dict[reaction_name]
            #print (hsa_gene_list)
            #print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))
            for gene in hsa_gene_list:
                if (gene in common_gene_list):
                #print (j)
                    #print (gene)
                #sample_node_feature.loc[j,gene] = exp_dict[gene][index] if gene in exp_dict else 0
                    sample_node_feature.at[j,gene] = GTEx_exp_match.at[index,gene]
                    #print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))
                else:
                    continue
        else:
            pass
        #break
    nodes_features.append(sample_node_feature)
    print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))
    #break
print ("store all sample node features done")
print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))

['ANKH']
True
2023-09-11  10:45:43
0
2023-09-11  10:47:11
1
2023-09-11  10:48:40
2


KeyboardInterrupt: 

In [211]:
# 清理内存
del GTEx_exp
del zero_matrix


In [220]:
print (nodes_features)

[            Reaction             Type  IGKV2-28  IGKV2D-30  TRAV19  IGKV3D-20  \
0      R-HSA-5216072     EwasReaction       0.0        0.0     0.0        0.0   
1      R-HSA-5226964  ProteinReaction       0.0        0.0     0.0        0.0   
2      R-HSA-5215980     EwasReaction       0.0        0.0     0.0        0.0   
3      R-HSA-5339528  ProteinReaction       0.0        0.0     0.0        0.0   
4      R-HSA-5339529     EwasReaction       0.0        0.0     0.0        0.0   
...              ...              ...       ...        ...     ...        ...   
13584  R-HSA-1247665  ProteinReaction       0.0        0.0     0.0        0.0   
13585  R-HSA-8870469  ProteinReaction       0.0        0.0     0.0        0.0   
13586  R-HSA-8870467  ComplexReaction       0.0        0.0     0.0        0.0   
13587  R-HSA-8870461  ProteinReaction       0.0        0.0     0.0        0.0   
13588  R-HSA-8870455  ComplexReaction       0.0        0.0     0.0        0.0   

       IGKV1-12  LCE6A  TM

In [205]:
print (sample_node_feature.iloc[0:10,0:10])
a =sample_node_feature.iloc[:,2:sample_node_feature.shape[1]]
a["sum"]=a.apply(lambda x:sum(x),axis=1)
print (a["sum"])

        Reaction             Type  IGKV2-28  IGKV2D-30  TRAV19  IGKV3D-20  \
0  R-HSA-5216072     EwasReaction       0.0        0.0     0.0        0.0   
1  R-HSA-5226964  ProteinReaction       0.0        0.0     0.0        0.0   
2  R-HSA-5215980     EwasReaction       0.0        0.0     0.0        0.0   
3  R-HSA-5339528  ProteinReaction       0.0        0.0     0.0        0.0   
4  R-HSA-5339529     EwasReaction       0.0        0.0     0.0        0.0   
5  R-HSA-5336453  ProteinReaction       0.0        0.0     0.0        0.0   
6  R-HSA-5336426             None       0.0        0.0     0.0        0.0   
7  R-HSA-5336454  ProteinReaction       0.0        0.0     0.0        0.0   
8  R-HSA-5216137     EwasReaction       0.0        0.0     0.0        0.0   
9  R-HSA-8932851  ProteinReaction       0.0        0.0     0.0        0.0   

   IGKV1-12  LCE6A  TMEM129  E2F8  
0       0.0    0.0      0.0   0.0  
1       0.0    0.0      0.0   0.0  
2       0.0    0.0      0.0   0.0  
3       

In [222]:
def getsizeof(p_object, default=None): # real signature unknown; restored from __doc__
    """
    getsizeof(object [, default]) -> int
    
    Return the size of object in bytes.
    """
    return 0
def binary_conversion(var: int):
    """
    二进制单位转换
    :param var: 需要计算的变量，bytes值
    :return: 单位转换后的变量，kb 或 mb
    """
    assert isinstance(var, int)
    if var <= 1024:
        return f'占用 {round(var / 1024, 2)} KB内存'
    else:
        return f'占用 {round(var / (1024 ** 2), 2)} MB内存'
from sys import getsizeof as getsize
print (binary_conversion(getsize(nodes_features)))
#keys = dir()
#for variable in keys:
#    print(variable, binary_conversion(getsize(eval(variable))), '\n')

占用 0.18 KB内存


In [83]:
def  load_data(nodes="/public/home/liujunwu/workdir/scripts/GNN_Reactome/test_data/Cora/Cora/cora.content",edges="/public/home/liujunwu/workdir/scripts/GNN_Reactome/test_data/Cora/Cora/cora.cites"):
    idx_features_labels = np.genfromtxt("{}".format(nodes),dtype=np.dtype(str))# 读取节点特征和标签
    #print (idx_features_labels[:,-1])
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32) # 读取节点特征
    dict = {int(element):i for i,element in enumerate(idx_features_labels[:, 0:1].reshape(-1))}    # 建立字典
    labels = encode_onehot(idx_features_labels[:, -1])                       # 节点标签用onehot方式表示
    e = np.genfromtxt("{}".format(edges), dtype=np.int32)    # 读取边信息
    edges = []
    for i, x in enumerate(e):
        edges.append([dict[e[i][0]], dict[e[i][1]]])                         # 若A->B有边 则B->A 也有边   ### 后续这里要修改？需要是有向图                 
        edges.append([dict[e[i][1]], dict[e[i][0]]])                         # 给的数据是没有从0开始需要转换
    features = normalize(features)                                           # 特征值归一化       
    features = torch.tensor(np.array(features.todense()), dtype=torch.float32)
    labels = torch.LongTensor(np.where(labels)[1])                       
    edges = torch.tensor(edges, dtype=torch.int64).T
    return features, edges, labels
features, edges, labels = load_data()                         
idx_train = range(2000)                                       # 其中2000个点是训练数据                   
idx_test = range(2000, 2700)                                  # 700个测试数据
idx_train = torch.LongTensor(idx_train) # 用于表示包含整数（64整型数据）的张量
idx_test = torch.LongTensor(idx_test)
#print (idx_test.tolist()) 
print (labels[0])

tensor(3)


In [91]:
from torch.nn import Linear

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        print (hidden_channels)
        print (type(dataset.num_classes))
        #print (dataset.num_classes.dtype)
        self.lin = Linear(hidden_channels, dataset.num_classes)
        
    def forward(self, x, edge_index, batch):
        # 1. 获得节点嵌入
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        
        # 2. Readout layer
        x = global_mean_pool(x, batch)   # [batch_size, hidden_channels]
        
        # 3. 分类器
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)  # 节点特征维度，隐藏神经元个数，标签数目

64
<class 'int'>


In [62]:
model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    
    for data in train_loader:
        optimizer.zero_grad()
        
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y)

        loss.backward()
        optimizer.step()

def test(loader):
    model.eval()
    
    correct = 0
    for data in loader:                            # 批遍历测试集数据集。
        out = model(data.x, data.edge_index, data.batch) # 一次前向传播
        pred = out.argmax(dim=1)                         # 使用概率最高的类别
        correct += int((pred == data.y).sum())           # 检查真实标签
    return correct / len(loader.dataset)


In [ ]:
def train():
    model.train()
    
    for data in train_loader:
        optimizer.zero_grad()
        
        out = model(data.x, data.edge_index)
        loss = criterion(out, data.y)

        loss.backward()
        optimizer.step()

def test(loader):
    model.eval()
    
    correct = 0
    for data in loader:                            # 批遍历测试集数据集。
        out = model(data.x, data.edge_index, data.batch) # 一次前向传播
        pred = out.argmax(dim=1)                         # 使用概率最高的类别
        correct += int((pred == data.y).sum())           # 检查真实标签
    return correct / len(loader.dataset)

In [93]:
class GCN(torch.nn.Module):
    def __init__(self,feature,hidden,classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(feature, hidden)  #输入=节点特征维度，hidden是中间隐藏神经元个数
        self.conv2 = GCNConv(hidden, classes)
    def forward(self, features, edges):
        features = self.conv1(features, edges)
        features = F.relu(features)
        features = F.dropout(features, training=self.training)
        features = self.conv2(features, edges)
        return F.log_softmax(features, dim=1)

In [94]:
# 设置参数和模型定义
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(1433, 16, 7)                                               # 每一个节点有 1433个特征                                      
optimizer = torch.optim.Adam(model.parameters(),lr=0.01,weight_decay=5e-4)               # 梯度优化算法

In [97]:
#模型训练
model.train()
for epoch in range(300):
    optimizer.zero_grad()
    out = model(features, edges)    #模型的输入有节点特征还有边特征,使用的是全部数据
    loss = F.nll_loss(out[idx_train], labels[idx_train])    #损失仅仅计算的是训练集的损失
    loss.backward()
    optimizer.step()
    if (epoch > 250):
        print(f"epoch:{epoch+1}, loss:{loss.item()}")
    else:
        pass



epoch:252, loss:0.3696596324443817
epoch:253, loss:0.364935964345932
epoch:254, loss:0.3683640658855438
epoch:255, loss:0.3657892048358917
epoch:256, loss:0.3638060390949249
epoch:257, loss:0.3660072982311249
epoch:258, loss:0.36475828289985657
epoch:259, loss:0.36472803354263306
epoch:260, loss:0.36476242542266846
epoch:261, loss:0.36531245708465576
epoch:262, loss:0.3770420253276825
epoch:263, loss:0.3715515434741974
epoch:264, loss:0.37264105677604675
epoch:265, loss:0.36958810687065125
epoch:266, loss:0.36638206243515015
epoch:267, loss:0.3630881905555725
epoch:268, loss:0.3794698417186737
epoch:269, loss:0.369625061750412
epoch:270, loss:0.3701733648777008
epoch:271, loss:0.3751410245895386
epoch:272, loss:0.36737310886383057
epoch:273, loss:0.36584511399269104
epoch:274, loss:0.378248929977417
epoch:275, loss:0.37657520174980164
epoch:276, loss:0.3594202995300293
epoch:277, loss:0.3686261773109436
epoch:278, loss:0.3610154688358307
epoch:279, loss:0.3707183599472046
epoch:280, lo

In [99]:
#测试：
model.eval()
_,pred = model(features,edges).max(dim=1)

correct = pred[idx_test].eq(labels[idx_test]).sum()            # 计算预测与标签相等个数
acc = int(correct) / int(len(idx_test))                        # 计算正确率
print(acc)
# 另一种写法
'''
test_predict = model(data.x, data.edge_index)[data.test_mask]
max_index = torch.argmax(test_predict, dim=1)
test_true = data.y[data.test_mask]
correct = 0
for i in range(len(max_index)):
    if max_index[i] == test_true[i]:
        correct += 1
print('测试集准确率为：{}%'.format(correct*100/len(test_true)
'''

0.8557142857142858


"\ntest_predict = model(data.x, data.edge_index)[data.test_mask]\nmax_index = torch.argmax(test_predict, dim=1)\ntest_true = data.y[data.test_mask]\ncorrect = 0\nfor i in range(len(max_index)):\n    if max_index[i] == test_true[i]:\n        correct += 1\nprint('测试集准确率为：{}%'.format(correct*100/len(test_true)\n"

In [50]:
from torch_geometric.datasets import TUDataset
dataset = TUDataset('data/TUDataset', name = 'MUTAG')
data = dataset[0]

Extracting data/TUDataset/MUTAG/MUTAG.zip
Processing...
Done!


In [93]:
from torch_geometric.loader import DataLoader
print (type(dataset))
train_dataset = dataset[:150]
test_dataset = dataset[150:]
#print (type(data))
print (type(data.y))
#print (data.edge_index)
#print (data.batch)
#train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
#print (type(train_loader))

<class 'torch_geometric.datasets.tu_dataset.TUDataset'>
<class 'torch.Tensor'>


In [81]:
model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()
from torch_geometric.nn import global_mean_pool
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
def train():
    model.train()
    
    for data in train_loader:
        optimizer.zero_grad()
        
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y)

        loss.backward()
        optimizer.step()

def test(loader):
    model.eval()
    
    correct = 0
    for data in loader:                            # 批遍历测试集数据集。
        out = model(data.x, data.edge_index, data.batch) # 一次前向传播
        pred = out.argmax(dim=1)                         # 使用概率最高的类别
        correct += int((pred == data.y).sum())           # 检查真实标签
    return correct / len(loader.dataset)

for epoch in range(1, 121):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Train Acc: 0.6600, Test Acc: 0.6842
Epoch: 002, Train Acc: 0.6600, Test Acc: 0.6842
Epoch: 003, Train Acc: 0.6600, Test Acc: 0.6842
Epoch: 004, Train Acc: 0.6600, Test Acc: 0.6842
Epoch: 005, Train Acc: 0.6667, Test Acc: 0.6842
Epoch: 006, Train Acc: 0.6733, Test Acc: 0.7105
Epoch: 007, Train Acc: 0.7267, Test Acc: 0.7105
Epoch: 008, Train Acc: 0.7267, Test Acc: 0.7105
Epoch: 009, Train Acc: 0.7267, Test Acc: 0.7368
Epoch: 010, Train Acc: 0.7333, Test Acc: 0.7105
Epoch: 011, Train Acc: 0.7467, Test Acc: 0.6842
Epoch: 012, Train Acc: 0.7533, Test Acc: 0.7105
Epoch: 013, Train Acc: 0.7467, Test Acc: 0.6842
Epoch: 014, Train Acc: 0.7533, Test Acc: 0.6842
Epoch: 015, Train Acc: 0.7667, Test Acc: 0.6842
Epoch: 016, Train Acc: 0.7533, Test Acc: 0.6842
Epoch: 017, Train Acc: 0.7467, Test Acc: 0.7368
Epoch: 018, Train Acc: 0.7667, Test Acc: 0.6842
Epoch: 019, Train Acc: 0.7600, Test Acc: 0.6842
Epoch: 020, Train Acc: 0.7467, Test Acc: 0.7105
Epoch: 021, Train Acc: 0.7600, Test Acc:

In [87]:
print (model)

GCN(
  (conv1): GCNConv(7, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [92]:
print (type(train_dataset[0].y))
print (train_dataset[0].x.dtype)
print (train_dataset[0].y.dtype)
print (dataset[0].edge_index.dtype)

<class 'torch.Tensor'>
torch.float32
torch.int64
torch.int64
